In [ ]:
import os
import time
import typing
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Concatenate,
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    PointCloud,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
    ConstantVectorSource
)

from manipulation import running_as_notebook
# from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation, AddPointClouds
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph

from utils import *
from perception_utils import perceive

In [ ]:
meshcat = StartMeshcat()

In [ ]:
drivers = f"""
model_drivers:
    iiwa: !IiwaDriver
        control_mode: position_only
        hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}

cameras:
    camera0:
        name: camera0
        depth: true
        X_PB:
            base_frame: camera0::base

    camera1:
        name: camera1
        depth: true
        X_PB:
            base_frame: camera1::base

    camera2:
        name: camera2
        depth: true
        X_PB:
            base_frame: camera2::base
"""

directives = f"""
directives:
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
        default_joint_positions:
            iiwa_joint_1: [-1.57]
            iiwa_joint_2: [0.1]
            iiwa_joint_3: [0]
            iiwa_joint_4: [-1.2]
            iiwa_joint_5: [0]
            iiwa_joint_6: [ 1.6]
            iiwa_joint_7: [0]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0
        X_PC:
            translation: [0, -0.5, 0]
            rotation: !Rpy {{ deg: [0, 0, 180] }}
    - add_model:
        name: wsg
        file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: wsg::body
        X_PC:
            translation: [0, 0, 0.09]
            rotation: !Rpy {{deg: [90, 0, 90]}}
    - add_model:
        name: floor
        file: package://stackbot/floor.sdf
    - add_weld:
        parent: world
        child: floor::floor_link
        X_PC:
            translation: [0, 0, -0.05]
    - add_model:
        name: platform
        file: package://stackbot/platform.sdf
    - add_weld:
        parent: world
        child: platform::platform_link
"""

block_numbers = np.random.choice(range(11), size=np.random.randint(4, 7), replace=False)
print("This scenario uses blocks:", block_numbers)
blocks = [f"block{i}" for i in block_numbers]

block_directives = create_randomized_block_directives(blocks)

This scenario uses blocks: [ 8  4 10  7]


In [ ]:
# Add cameras to the scenario
camera_directives = f"""
    - add_frame:
        name: camera0_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [-120.0, 0.0, 180.0]}}
            translation: [0, 1.0, 0.8]

    - add_model:
        name: camera0
        file: package://manipulation/camera_box.sdf

    - add_weld:
        parent: camera0_origin
        child: camera0::base

    - add_frame:
        name: camera1_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [-125, 0.0, 90.0]}}
            translation: [1.0, 0.1, 0.8]

    - add_model:
        name: camera1
        file: package://manipulation/camera_box.sdf

    - add_weld:
        parent: camera1_origin
        child: camera1::base

    - add_frame:
        name: camera2_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [-120.0, 0.0, -90.0]}}
            translation: [-1.0, 0.1, 0.8]

    - add_model:
        name: camera2
        file: package://manipulation/camera_box.sdf

    - add_weld:
        parent: camera2_origin
        child: camera2::base
"""

In [ ]:
scenario_yaml = drivers + directives + block_directives + camera_directives

In [ ]:
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(
    scenario=scenario,
    meshcat=meshcat,
    package_xmls=["../assets/models/package.xml"],
)
plant = station.GetSubsystemByName("plant")

builder = DiagramBuilder()
builder.AddSystem(station)

# Connect initial positions to builder
force_limit = builder.AddSystem(ConstantVectorSource([60.0]))
builder.Connect(force_limit.get_output_port(), station.GetInputPort("wsg.force_limit"))

initial_iiwa_pos = builder.AddSystem(ConstantVectorSource(np.zeros(station.GetInputPort("iiwa.position").size())))
builder.Connect(initial_iiwa_pos.get_output_port(), station.GetInputPort("iiwa.position"))

initial_gripper_pos = builder.AddSystem(ConstantVectorSource([0.0]))
builder.Connect(initial_gripper_pos.get_output_port(), station.GetInputPort("wsg.position"))

# Add the point clouds to the diagram and export to builder
pointClouds = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)
builder.ExportOutput(pointClouds["camera0"].get_output_port(), "camera0_point_cloud")
builder.ExportOutput(pointClouds["camera1"].get_output_port(), "camera1_point_cloud")
builder.ExportOutput(pointClouds["camera2"].get_output_port(), "camera2_point_cloud")

diagram = builder.Build()


In [ ]:
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
simulator = Simulator(diagram, diagram_context)

# run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5)
meshcat.StopRecording()
meshcat.PublishRecording()

In [ ]:
# Evaluate all camera outputs
camera0_point_cloud = diagram.GetOutputPort("camera0_point_cloud").Eval(diagram_context)
camera1_point_cloud = diagram.GetOutputPort("camera1_point_cloud").Eval(diagram_context)
camera2_point_cloud = diagram.GetOutputPort("camera2_point_cloud").Eval(diagram_context)

# concatenate the point clouds
point_cloud = Concatenate([camera0_point_cloud, camera1_point_cloud, camera2_point_cloud])

# downsample the point clouds
point_cloud = point_cloud.VoxelizedDownSample(0.01)


In [10]:
# list of (pose, length) sorted in descending order of length
X_WBs_by_length = perceive(point_cloud, meshcat)

<class 'pydrake.perception.PointCloud'>
<class 'pydrake.perception.PointCloud'>
<class 'pydrake.perception.PointCloud'>
<class 'pydrake.perception.PointCloud'>
<class 'pydrake.perception.PointCloud'>
